Colab에서 이 노트북을 연 경우, 필요한 라이브러리를 미리 설치해야 할 가능성이 높습니다. 아래 코드의 주석 (#)을 해제하고 실행합니다.

In [1]:
! pip install datasets transformers[sentencepiece]

로컬에서 이 노트북을 연 경우 (Jypyter Notebook 등) 최신 버전의 Transformer 설치가 완료된 가상환경을 사용해야 합니다. 
머신러닝딥러닝 과목 실습 시간에 사용한 가상환경을 베이스로 위 코드를 통해 transformer / transformer dataset을 설치해서 사용할 수도 있습니다. 

# Training your own tokenizer from scratch

이번 실습을 통해, 주어진 corpus를 활용해서 tokenizer를 fine-tune하는 방법을 실습해봅니다. 이를 활용하여 Language Model을 학습시킬 수 있습니다.

Tokenizer를 *학습*하는것이 필요한 이유는, Transformer 모델은 대부분 subword tokenization algorithm을 사용하기 때문입니다. subword tokenization algorithm은 사용하는 corpus에서 자주 등장하는 subword를 분류하도록 tokenizer를 학습 시킵니다.

추가적인 읽을거리로 Huggingface [공식 학습 자료](https://huggingface.co/course/chapter2/4?fw=pt)에서 tokenizer에 대한 전반적인 내용을 열람하고, [tokenizers summary](https://huggingface.co/transformers/tokenizer_summary.html) 챕터에서 다양한 subword tokenization algorithm을 접할 수 있습니다. 

In [46]:
!pip install torch-gpu

ERROR: Could not find a version that satisfies the requirement torch-gpu (from versions: none)
ERROR: No matching distribution found for torch-gpu


In [44]:
import torch

In [45]:
torch.cuda.is_available()

False

## Getting a corpus

tokenizer 학습을 위해 text 데이터가 필요합니다. Huggingface [Datasets](https://github.com/huggingface/datasets) 라이브러리의 `load_dataset` 함수를 활용해서 데이터를 다운로드 받을 수 있습니다. 이는 다음 실습시간과 AI 설계 과정에서도 유용하게 사용할 수 있습니다.

In [2]:
from datasets import load_dataset

이번 실습시간에서는 Wikitext-2 dataset 을 사용해서 tokenizer를 학습합니다. 빠른 실습 진행을 위해 4.5MB 정도로 작은 데이터를 사용했지만 필요에 따라 훨씬 큰 (수백GB 정도의) 데이터를 사용할 수 있고, 영어 뿐만 아니라 다양한 언어로 된 텍스트를 사용할 수도 있습니다. 여러 언어가 섞인 데이터를 사용할 수도 있습니다.


In [3]:
dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to C:/Users/user/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


We can have a look at the dataset, which as 36,718 texts:

In [4]:
dataset

Dataset({
    features: ['text'],
    num_rows: 36718
})

배열의 원소에 접근하듯이 단일 데이터에 접근할 수 있습니다.

In [5]:
dataset[1]

{'text': ' = Valkyria Chronicles III = \n'}

배열의 부분배열을 호출하듯이, dataset의 일부를 간단히 접근할 수 있습니다. 아래 코드를 실행하면 `"text"` 가 key이고, 5개의 문장 list를 value로 가지고 있는 dictionary를 열람할 수 있습니다.

In [6]:
dataset[:5]

{'text': ['',
  ' = Valkyria Chronicles III = \n',
  '',
  ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n',
  " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making th

Tokenizer를 학습시키기 위한 Huggingface API는 batch text를 출력하는 iterator를 요구합니다. 예시로 아래와 같이 텍스트 list의 list를 사용할 수 있습니다.

In [7]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

모든 데이터를 한번에 메모리에 올리는 것을 방지하기 위해 Python Iterator를 정의하여 필요한 데이터만 메모리에 올리도록 합니다. 이는 거대한 데이터셋을 사용하여 학습할 때 유용하게 쓸 수 있습니다.

In [8]:
def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

## Using an existing tokenizer

기존에 존재하는 tokenizer와 동일한 알고리즘과 파라미터를 사용하여 tokenizer를 학습하고자 한다면, `train_new_from_iterator` API를 사용할 수 있습니다. 예시로, BERT Tokenizer를 wikitext-2 데이터셋을 통해 학습시켜봅시다.

먼저, 사용하고자 하는 tokenizer를 load합니다.

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

C:\Users\user\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Load된 tokenizer가 *fast* 버전 (Huggingface backbone)인지 확인합니다. 아니라면 하단의 코드가 작동하지 않습니다.

In [10]:
tokenizer.is_fast

True

이제 앞서 정의한 traning corpus (list의 list 혹은 앞서 정의한 iterator)를 `train_new_from_iterator` 메서드에 입력합니다. 이때 tokenizer가 가질 vocabulary size를 정할 수 있습니다. 

In [11]:
new_tokenizer = tokenizer.train_new_from_iterator(batch_iterator(), vocab_size=25000)

Tokenizer 학습이 완료되었습니다! 아래와 같이 테스트를 해볼 수 있습니다:

In [12]:
for i in range(5):
  print("Original Text:{}".format(dataset[i]["text"]))
print("Tokenized:")
new_tokenizer(dataset[:5]["text"])

Original Text:
Original Text: = Valkyria Chronicles III = 

Original Text:
Original Text: Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . 

Original Text: The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent mul

{'input_ids': [[2, 3], [2, 33, 9773, 10627, 4171, 33, 3], [2, 3], [2, 5693, 1067, 1092, 2045, 9773, 23, 30, 1843, 7499, 18065, 1025, 10627, 12, 3923, 30, 810, 758, 617, 1289, 1290, 1129, 1291, 1292, 1293, 1294, 1040, 16, 7591, 18, 9773, 1506, 1495, 4737, 3191, 23, 13, 16, 6295, 4419, 1515, 1546, 9773, 10627, 4171, 3739, 4353, 16, 1559, 69, 15148, 2714, 36, 17, 36, 3527, 2677, 1898, 3106, 1565, 14932, 1015, 1513, 8708, 18, 20509, 1540, 1495, 5956, 16675, 18, 20918, 1509, 2555, 2846, 1509, 4353, 16, 1620, 1559, 1495, 2588, 1898, 1509, 1495, 9773, 2061, 18, 3729, 3012, 1511, 1495, 2271, 10783, 1506, 15148, 1513, 2776, 36, 17, 36, 1814, 7707, 1546, 1699, 15512, 16, 1495, 2642, 4551, 8086, 1515, 1495, 1697, 1898, 1513, 5862, 1495, 6, 23073, 6, 16, 69, 22752, 2893, 4963, 6783, 1495, 5420, 1506, 5831, 1668, 1890, 1495, 4699, 12085, 1019, 2273, 1772, 2117, 5133, 3134, 3697, 1513, 1636, 3275, 1881, 2100, 1495, 8786, 4963, 6, 3171, 3481, 2274, 23306, 6, 18, 3], [2, 1535, 1898, 2183, 3130, 1509, 

학습이 완료된 tokenizer를 `save_pretrained` 메서드로 로컬로 저장할 수 있습니다.

In [13]:
new_tokenizer.save_pretrained("my-new-tokenizer")

('my-new-tokenizer\\tokenizer_config.json',
 'my-new-tokenizer\\special_tokens_map.json',
 'my-new-tokenizer\\vocab.txt',
 'my-new-tokenizer\\added_tokens.json',
 'my-new-tokenizer\\tokenizer.json')

아래 블럭은 Huggingface 에서 제공하는 tokenizer 관련 튜토리얼 중, 제로 베이스에서 tokenizer를 학습 시키는 방법에 대한 내용입니다. 

## Building your tokenizer from scratch

To understand how to build your tokenizer from scratch, we have to dive a little bit more in the 🤗 Tokenizers library and the tokenization pipeline. This pipeline takes several steps:

- **Normalization**: Executes all the initial transformations over the initial input string. For example when you need to lowercase some text, maybe strip it, or even apply one of the common unicode normalization process, you will add a Normalizer.
- **Pre-tokenization**: In charge of splitting the initial input string. That's the component that decides where and how to pre-segment the origin string. The simplest example would be to simply split on spaces.
- **Model**: Handles all the sub-token discovery and generation, this is the part that is trainable and really dependent of your input data.
- **Post-Processing**: Provides advanced construction features to be compatible with some of the Transformers-based SoTA models. For instance, for BERT it would wrap the tokenized sentence around [CLS] and [SEP] tokens.

And to go in the other direction:

- **Decoding**: In charge of mapping back a tokenized input to the original string. The decoder is usually chosen according to the `PreTokenizer` we used previously.

For the training of the model, the 🤗 Tokenizers library provides a `Trainer` class that we will use.

All of these building blocks can be combined to create working tokenization pipelines. To give you some examples, we will show three full pipelines here: how to replicate GPT-2, BERT and T5 (which will give you an example of BPE, WordPiece and Unigram tokenizer).

### WordPiece model like BERT

Let's have a look at how we can create a WordPiece tokenizer like the one used for training BERT. The first step is to create a `Tokenizer` with an empty `WordPiece` model:

In [14]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

tokenizer = Tokenizer(models.WordPiece(unl_token="[UNK]"))

This `tokenizer` is not ready for training yet. We have to add some preprocessing steps: the normalization (which is optional) and the pre-tokenizer, which will split inputs into the chunks we will call words. The tokens will then be part of those words (but can't be larger than that).

In the case of BERT, the normalization is lowercasing. Since BERT is such a popular model, it has its own normalizer:

In [15]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

If you want to customize it, you can use the existing blocks and compose them in a sequence: here for instance we lower case, apply NFD normalization and strip the accents:

In [16]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

There is also a `BertPreTokenizer` we can use directly. It pre-tokenizes using white space and punctuation:

In [17]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

Like for the normalizer, we can combine several pre-tokenizers in a `Sequence`. If we want to have a quick look at how it preprocesses the inputs, we can call the `pre_tokenize_str` method:

In [18]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('This', (0, 4)),
 ('is', (5, 7)),
 ('an', (8, 10)),
 ('example', (11, 18)),
 ('!', (18, 19))]

Note that the pre-tokenizer not only split the text into words but keeps the offsets, that is the beginning and start of each of those words inside the original text. This is what will allow the final tokenizer to be able to match each token to the part of the text that it comes from (a feature we use for question answering or token classification tasks).

We can now train our tokenizer (the pipeline is not entirely finished but we will need a trained tokenizer to build the post-processor), we use a `WordPieceTrainer` for that. The key thing to remember is to pass along the special tokens to the trainer, as they won't be seen in the corpus.

In [19]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

To actually train the tokenizer, the method looks like what we used before: we can either pass some text files, or an iterator of batches of texts:

In [20]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

Now that the tokenizer is trained, we can define the post-processor: we need to add the CLS token at the beginning and the SEP token at the end (for single sentences) or several SEP tokens (for pairs of sentences). We use a [`TemplateProcessing`](https://huggingface.co/docs/tokenizers/python/latest/api/reference.html#tokenizers.processors.TemplateProcessing) to do this, which requires to know the IDs of the CLS and SEP token (which is why we waited for the training).

So let's first grab the ids of the two special tokens:

In [21]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


And here is how we can build our post processor. We have to indicate in the template how to organize the special tokens with one sentence (`$A`) or two sentences (`$A` and `$B`). The `:` followed by a number indicates the token type ID to give to each part.

In [22]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

We can check we get the expected results by encoding a pair of sentences for instance:

In [23]:
encoding = tokenizer.encode("This is one sentence.", "With this one we have a pair.")

We can look at the tokens to check the special tokens have been inserted in the right places:

In [24]:
encoding.tokens

['[CLS]',
 'this',
 'is',
 'one',
 'sentence',
 '.',
 '[SEP]',
 'with',
 'this',
 'one',
 'we',
 'have',
 'a',
 'pair',
 '.',
 '[SEP]']

And we can check the token type ids are correct:

In [25]:
encoding.type_ids

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]

The last piece in this tokenizer is the decoder, we use a `WordPiece` decoder and indicate the special prefix `##`:

In [26]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

Now that our tokenizer is finished, we need to wrap it inside a Transformers object to be able to use it with the Transformers library. More specifically, we have to put it inside the class of tokenizer fast corresponding to the model we want to use, here a `BertTokenizerFast`:

In [27]:
from transformers import BertTokenizerFast

new_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

And like before, we can use this tokenizer as a normal Transformers tokenizer, and use the `save_pretrained` or `push_to_hub` methods.

If the tokenizer you are building does not match any class in Transformers because it's really special, you can wrap it in `PreTrainedTokenizerFast`.

### BPE model like GPT-2

Let's now have a look at how we can create a BPE tokenizer like the one used for training GPT-2. The first step is to create a `Tokenizer` with an empty `BPE` model:

In [28]:
tokenizer = Tokenizer(models.BPE())

Like before, we have to add the optional normalization (not used in the case of GPT-2) and we need to specify a pre-tokenizer before training. In the case of GPT-2, the pre-tokenizer used is a byte level pre-tokenizer:

In [29]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

If we want to have a quick look at how it preprocesses the inputs, we can call the `pre_tokenize_str` method:

In [30]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('This', (0, 4)),
 ('Ġis', (4, 7)),
 ('Ġan', (7, 10)),
 ('Ġexample', (10, 18)),
 ('!', (18, 19))]

We used the same default as for GPT-2 for the prefix space, so you can see that each word gets an initial `'Ġ'` added at the beginning, except the first one.

We can now train our tokenizer! This time we use a `BpeTrainer`.

In [31]:
trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

To finish the whole pipeline, we have to include the post-processor and decoder:

In [32]:
tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)
tokenizer.decoder = decoders.ByteLevel()

And like before, we finish by wrapping this in a Transformers tokenizer object:

In [33]:
from transformers import GPT2TokenizerFast

new_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

### Unigram model like Albert

Let's now have a look at how we can create a Unigram tokenizer like the one used for training T5. The first step is to create a `Tokenizer` with an empty `Unigram` model:

In [34]:
tokenizer = Tokenizer(models.Unigram())

Like before, we have to add the optional normalization (here some replaces and lower-casing) and we need to specify a pre-tokenizer before training. The pre-tokenizer used is a `Metaspace` pre-tokenizer: it replaces all spaces by a special character (defaulting to ▁) and then splits on that character.

In [35]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.Replace("``", '"'), normalizers.Replace("''", '"'), normalizers.Lowercase()]
)
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

If we want to have a quick look at how it preprocesses the inputs, we can call the `pre_tokenize_str` method:

In [36]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

[('▁This', (0, 4)), ('▁is', (4, 7)), ('▁an', (7, 10)), ('▁example!', (10, 19))]

You can see that each word gets an initial `▁` added at the beginning, as is usually done by sentencepiece.

We can now train our tokenizer! This time we use a `UnigramTrainer`."We have to explicitely set the unknown token in this trainer otherwise it will forget it afterward.

In [37]:
trainer = trainers.UnigramTrainer(vocab_size=25000, special_tokens=["[CLS]", "[SEP]", "<unk>", "<pad>", "[MASK]"], unk_token="<unk>")
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

To finish the whole pipeline, we have to include the post-processor and decoder. The post-processor is very similar to what we saw with BERT, the decoder is just `Metaspace`, like for the pre-tokenizer.

In [38]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")

In [39]:
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS]:0 $A:0 [SEP]:0",
    pair="[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)
tokenizer.decoder = decoders.Metaspace()

And like before, we finish by wrapping this in a Transformers tokenizer object:

In [40]:
from transformers import AlbertTokenizerFast

new_tokenizer = AlbertTokenizerFast(tokenizer_object=tokenizer)

## Use your new tokenizer to train a language model!

You can either use your new tokenizer in the language modeling from scratch notebook [Link to come] or use the `--tokenizer_name` argument in the [language modeling scripts](https://github.com/huggingface/transformers/tree/master/examples/pytorch/language-modeling) to use it there to train a model from scratch.